<a href="https://colab.research.google.com/github/JNJ25802058/Conversational-Chatbot/blob/main/Copy_of_Retrieval_and_Generation_8_May_2025_1000_Page_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

In [ ]:
!pip install tensorflow==2.9.0 -qq
!pip install transformers==4.20.0 -qq
!pip install sentence-transformers -qq
!pip install faiss-cpu  -qq

ERROR: Could not find a version that satisfies the requirement tensorflow==2.9.0 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0)
ERROR: No matching distribution found for tensorflow==2.9.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.3/77.3 kB 955.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.7/220.7 kB 7.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 13.8 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for outp

In [ ]:
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import faiss
import os
import re
from google.colab import userdata
from huggingface_hub import login
from google.colab import drive
import json
#___________________________________________________________________________

drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/AP')

HF_TOKEN = userdata.get('HF_TOKEN')
login(token=HF_TOKEN)

file_name = 'UTD_Knowledge_Base2.json'
directory = '/content/drive/MyDrive/AP'
file_path = os.path.join(directory, file_name)

with open(file_path, 'r') as json_file:
    UTD_Knowledge_Base = json.load(json_file)

UTD_Knowledge_Base = {int(k): v for k, v in UTD_Knowledge_Base.items()}

#___________________________________________________________________________

Mounted at /content/drive


# Import File

In [ ]:
import pandas as pd

file_name = 'sample_chunks.xls'
directory = '/content/drive/MyDrive/AP'
file_path = os.path.join(directory, file_name)

df2 = pd.read_csv(file_path)


In [ ]:
df2.head()

,Unnamed: 0,url,chunk_id,text,embedding
0,2714,https://jindal.utdallas.edu/ms-hlm-faq,567_12,Graduate Certificate in Healthcare Information...,"[0.014793220907449722, 0.019371768459677696, -..."
1,6918,https://jindal.utdallas.edu/news/utd-top-100-b...,1586_7,The number of universities from the United Sta...,"[-0.0015820126282051206, -0.013461489230394363..."
2,10341,https://jindal.utdallas.edu/blog/expand-profes...,3385_7,While the website serves as a social networkin...,"[-0.02495254948735237, 7.718194865447003e-06, ..."
3,7305,https://jindal.utdallas.edu/news/ut-dallas-com...,1668_5,Undergraduate Admission\nGraduate Admission\nM...,"[-0.0043958136811852455, -0.012605947442352772..."
4,10949,https://jindal.utdallas.edu/blog/best-study-sp...,4049_8,Parks:Some fresh air and a change of environme...,"[0.0015906590269878507, -0.003994510509073734,..."


In [ ]:
from collections import defaultdict

# Start with an empty dict with lists to group chunks by URL
restructured_data = defaultdict(lambda: {"url": "", "chunks": []})

# Fill in the structure
for _, row in df2.iterrows():
    url = row['url']
    text = row['text']
    restructured_data[url]['url'] = url
    restructured_data[url]['chunks'].append(text)

# Convert to integer-keyed UTD_Knowledge_Base
UTD_Knowledge_Base = {i: v for i, v in enumerate(restructured_data.values())}


In [ ]:
UTD_Knowledge_Base

In [ ]:
# Load pre-trained SBERT model
similarity_model = SentenceTransformer('all-MiniLM-L6-v2')
#___________________________________________________________________________

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the LLM (example: Mistral-7B-Instruct)
model_name = "mistralai/Mistral-7B-Instruct-v0.1"  # Change to another model if needed
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
#___________________________________________________________________________

from pprint import pprint
import warnings
import textwrap
# Suppress specific warnings
warnings.filterwarnings("ignore", message=".*do_sample.*")
warnings.filterwarnings("ignore", message=".*pad_token_id.*")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-05864fe6284f>", line 11, in <cell line: 0>
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py", line 571, in from_pretrained
    return model_class.from_pretrained(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py", line 279, in _wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py", line 4380, in from_pretrained
    device_map = _get_device_map(model, dev

In [ ]:
import faiss
from sentence_transformers import SentenceTransformer
import re
import textwrap
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Initialize models
similarity_model = SentenceTransformer('all-MiniLM-L6-v2')
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [ ]:
import re
import faiss
import numpy as np

def sanitize_query(query):
    """Clean and normalize the query text while preserving structure."""
    # Replace newlines with space placeholders we can identify later
    query = query.replace('\n', ' NEWLINE_PLACEHOLDER ')

    # Remove other special characters and extra spaces
    query = re.sub(r'[^\w\s]', ' ', query)
    query = re.sub(r'\s+', ' ', query).strip()

    return query

def search_corpus(query, index, texts, urls, top_k=5, max_distance=1.0):
    """Search for relevant documents using vector similarity with improved relevance.

    Args:
        query: The search query text
        index: FAISS index for vector search
        texts: List of text documents
        urls: List of corresponding URLs
        top_k: Number of results to return
        max_distance: Maximum allowed distance (results with higher distance are excluded)
    """
    original_query = query
    query = sanitize_query(query)

    # print(f"Original query: '{original_query}'")
    # print(f"Sanitized query: '{query}'")

    # Get embeddings for the query
    query_embedding = similarity_model.encode([query], convert_to_numpy=True)

    # Search for many more candidates than we need to avoid missing relevant documents
    candidate_k = min(top_k * 10, len(texts))  # Get 10x more results initially
    distances, indices = index.search(query_embedding, candidate_k)

    # Process results with hybrid scoring
    results = []
    query_terms = set(query.lower().split())

    # Generate query variations for robust matching
    query_variations = [original_query.lower()]
    query_variations.append(' '.join(original_query.lower().split()))  # Normalized spaces
    query_variations.append(original_query.replace('\n', ' ').lower())  # Replace newlines

    # For partial matching - create key subphrases by removing one word at a time
    query_words = original_query.lower().split()
    if len(query_words) > 3:  # Only for queries with enough words
        for i in range(len(query_words)):
            # Skip one word and join the rest
            partial_query = ' '.join(query_words[:i] + query_words[i+1:])
            if len(partial_query) > len(original_query) * 0.7:  # Keep if at least 70% of original
                query_variations.append(partial_query)

    # Check for exact and near-exact matches first (prioritize them)
    exact_match_indices = []
    near_match_indices = []

    # Analyze all texts for matches
    for idx in range(len(texts)):
        text_lower = texts[idx].lower()
        text_normalized = ' '.join(text_lower.split())
        match_score = 0
        is_exact = False

        # Check for exact match with any query variation
        for qvar in query_variations[:3]:  # First 3 are exact variations
            if qvar in text_normalized or qvar in text_lower:
                exact_match_indices.append(idx)
                is_exact = True
                break

        if is_exact:
            continue

        # Check for partial matches with key subphrases
        for qvar in query_variations[3:]:  # These are partial variations
            if qvar in text_normalized or qvar in text_lower:
                near_match_indices.append(idx)
                break

        # Calculate percentage of query terms in the text
        matched_terms = sum(1 for term in query_terms if term in text_lower)
        term_percentage = matched_terms / len(query_terms) if query_terms else 0

        # If most terms match (>80%), consider it a near match as well
        if term_percentage > 0.8 and idx not in near_match_indices:
            near_match_indices.append(idx)

    # Process vector similarity results
    for i in range(len(indices[0])):
        idx = indices[0][i]
        if idx >= len(texts):  # Safety check
            continue

        text = texts[idx]
        distance = float(distances[0][i])

        # Skip contexts with distance greater than the threshold
        if distance > max_distance:
            continue

        # Calculate term overlap score with higher weight
        text_lower = text.lower()
        text_terms = set(text_lower.split())

        # Count query terms in the text (weighted by term importance)
        term_overlap = 0
        for term in query_terms:
            if term in text_lower:
                # Give higher weight to important terms (longer ones are usually more specific)
                term_weight = min(2.0, 0.5 + len(term)/10)
                term_overlap += term_weight

        # Normalize by sum of possible weights
        max_term_overlap = sum(min(2.0, 0.5 + len(term)/10) for term in query_terms)
        term_ratio = term_overlap / max_term_overlap if max_term_overlap > 0 else 0

        # Calculate key phrase presence with position-aware matching
        query_phrases = []
        query_words = query.lower().split()

        # Create sliding windows of words from the query
        for n in range(2, min(6, len(query_words) + 1)):  # phrases of length 2-5
            for i in range(len(query_words) - n + 1):
                phrase = " ".join(query_words[i:i+n])
                query_phrases.append((phrase, n/len(query_words)))  # (phrase, relative importance)

        # Check for phrase matches with weighting
        phrase_score = 0
        for phrase, importance in query_phrases:
            if phrase in text_lower:
                # Longer phrases and phrases at beginning/end are more important
                phrase_score += importance * 0.2

        phrase_bonus = min(0.5, phrase_score)  # Cap at 0.5

        # Calculate final relevance score (lower is better)
        # Apply much stronger bonuses for term overlap and phrase matches
        adjusted_distance = distance - (term_ratio * 0.5) - phrase_bonus

        # Apply bonuses for exact and near matches
        match_type = "none"
        if idx in exact_match_indices:
            adjusted_distance = -10.0  # Ensure exact matches come first
            match_type = "exact"
        elif idx in near_match_indices:
            adjusted_distance = -5.0 + (distance * 0.1)  # Near matches come next
            match_type = "near"

        results.append({
            "context": text,
            "url": urls[idx],
            "distance": distance,
            "adjusted_distance": adjusted_distance,
            "match_type": match_type
        })

    # Add any exact or near matches that weren't in the initial results
    for idx in exact_match_indices:
        # Check if already added
        if not any(r["url"] == urls[idx] and r["context"] == texts[idx] for r in results):
            # Get the actual distance for this document (if available)
            distance = 0.0  # Default to best distance
            for i in range(len(indices[0])):
                if indices[0][i] == idx:
                    distance = float(distances[0][i])
                    break

            # Only add if distance is below threshold
            if distance <= max_distance:
                results.append({
                    "context": texts[idx],
                    "url": urls[idx],
                    "distance": distance,
                    "adjusted_distance": -10.0,  # Ensure it comes first
                    "match_type": "exact"
                })

    for idx in near_match_indices:
        # Check if already added and not an exact match
        if not any(r["url"] == urls[idx] and r["context"] == texts[idx] for r in results) and idx not in exact_match_indices:
            # Get the actual distance for this document (if available)
            distance = 0.5  # Default to good distance
            for i in range(len(indices[0])):
                if indices[0][i] == idx:
                    distance = float(distances[0][i])
                    break

            # Only add if distance is below threshold
            if distance <= max_distance:
                results.append({
                    "context": texts[idx],
                    "url": urls[idx],
                    "distance": distance,
                    "adjusted_distance": -5.0,  # Better than vector matches, worse than exact
                    "match_type": "near"
                })

    # Sort by adjusted distance and take top_k
    results.sort(key=lambda x: x["adjusted_distance"])
    final_results = results[:top_k]

    # Debug output for match types
    match_types = {"exact": 0, "near": 0, "none": 0}
    for result in final_results:
        match_type = result.get("match_type", "none")
        match_types[match_type] += 1

        if match_type == "exact":
            print(f"EXACT MATCH FOUND: {result['context'][:100]}...")
        elif match_type == "near":
            print(f"NEAR MATCH FOUND: {result['context'][:100]}...")

    print(f"\nMatch statistics: {match_types['exact']} exact, {match_types['near']} near, {match_types['none']} vector-only")
    print(f"All results have distance <= {max_distance}")

    # Clean up the return structure for final output
    for result in final_results:
        del result["adjusted_distance"]
        if "match_type" in result:
            del result["match_type"]

    # # Debug output
    # print("\nRetrieved contexts:")
    # for i, ctx in enumerate(final_results):
    #     print(f"Context {i+1} (distance: {ctx['distance']:.4f}): {ctx['context'][:100]}... ({ctx['url']})")

    return final_results

def build_faiss_index(UTD_Knowledge_Base):
    """Build a FAISS index from the knowledge base with improved chunking."""
    texts, urls, original_texts = [], [], []

    # Process knowledge base
    for entry in UTD_Knowledge_Base.values():
        chunks = entry['chunks']

        # Add the title to each chunk for better context if available
        title = entry.get('title', '')
        if title:
            original_chunks = chunks.copy()
            chunks = []
            for chunk in original_chunks:
                # Only add title if it's not already at the beginning of the chunk
                if not chunk.startswith(title):
                    chunks.append(f"{title}\n{chunk}")
                else:
                    chunks.append(chunk)

        # Store both original and processed texts
        original_texts.extend(chunks)

        # Process chunks for embedding (with same sanitization as queries)
        processed_chunks = [sanitize_query(chunk) for chunk in chunks]
        texts.extend(processed_chunks)
        urls.extend([entry['url']] * len(chunks))

    # Convert text data into embeddings
    embeddings = similarity_model.encode(texts, convert_to_numpy=True)

    # Create FAISS index - using L2 norm for distance calculation
    dimension = embeddings.shape[1]

    # Normalize embeddings for improved similarity matching
    faiss.normalize_L2(embeddings)

    # Create an index that works with normalized vectors
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)

    # Return the index along with both original and processed texts
    return index, original_texts, urls

In [ ]:
def get_answer(chatbot, user_query):
    """Process a user query and return a response with sources."""
    response, sources = chatbot.generate_answer(user_query)
    wrapped_text = textwrap.fill(response, width=80)
    return response + sources

In [ ]:
def generate_from_llm(prompt, max_length=512, max_new_tokens=None, pad_token_id=None):
    """Generate text from the LLM based on the provided prompt.

    Args:
        prompt: The input prompt text
        max_length: Maximum total length including prompt and generated text
        max_new_tokens: Maximum number of new tokens to generate (overrides max_length if provided)
        pad_token_id: Token ID to use for padding

    Returns:
        Generated response text
    """
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4096).to(model.device)

    # Set up generation parameters
    generate_params = {
        **inputs,
        "temperature": 0.7,
        "top_p": 0.9,
        "do_sample": True
    }

    # Handle max_new_tokens and max_length correctly
    if max_new_tokens is not None:
        generate_params["max_new_tokens"] = max_new_tokens
    else:
        generate_params["max_length"] = max_length

    # Set pad_token_id if provided
    if pad_token_id is not None:
        generate_params["pad_token_id"] = pad_token_id

    # Generate text
    with torch.no_grad():
        output = model.generate(**generate_params)

    # Decode and extract the generated text
    full_output = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract the part after the prompt
    response = full_output[len(prompt):].strip()

    # Clean up any artifacts
    response = re.sub(r'^[^A-Za-z0-9]*', '', response)

    # If the response contains any potential unwanted format markers, clean them
    if "assistant:" in response.lower():
        response = response.split("assistant:", 1)[1].strip()

    return response

In [ ]:
class ConversationalChatbot:
    def __init__(self, index, texts, urls):
        self.index = index
        self.texts = texts
        self.urls = urls
        self.conversation_history = []
        self.max_history_length = 10  # Maximum number of turns to remember
        # Add common conversational patterns
        self.conversational_patterns = {
            "greetings": ["hi", "hello", "hey", "greetings", "good morning", "good afternoon", "good evening", "howdy"],
            "farewells": ["bye", "goodbye", "see you", "farewell", "talk to you later"],
            "thanks": ["thank you", "thanks", "appreciate it", "thank you very much"]
        }
        self.responses = {
            "greetings": ["Hello! How can I help you today?", "Hi there! What can I assist you with?", "Hey! What questions do you have?"],
            "farewells": ["Goodbye! Feel free to chat again if you have more questions.", "See you later! Have a great day!"],
            "thanks": ["You're welcome! Is there anything else I can help with?", "Happy to help! Let me know if you need anything else."]
        }

    def add_message(self, role, content):
        """Add a message to the conversation history."""
        self.conversation_history.append({"role": role, "content": content})
        # Trim history if it gets too long
        if len(self.conversation_history) > self.max_history_length * 2:
            self.conversation_history = self.conversation_history[-self.max_history_length*2:]

    def format_conversation_history(self):
        """Format the conversation history for the LLM prompt."""
        formatted_history = ""
        for msg in self.conversation_history:
            role = "Human" if msg["role"] == "user" else "Assistant"
            formatted_history += f"{role}: {msg['content']}\n\n"
        return formatted_history.strip()

    def is_conversational_query(self, query):
        """Check if the query is a basic conversational pattern like greeting."""
        query = query.lower().strip()

        # Check if query is ONLY a conversational pattern
        # Only match if the query is very short or consists primarily of the pattern
        if len(query.split()) <= 3:  # Only check short queries (3 words or less)
            for pattern_type, patterns in self.conversational_patterns.items():
                # Check for exact matches or if the entire query is contained in a pattern
                if query in patterns or any(pattern == query for pattern in patterns):
                    return pattern_type

                # For slightly longer queries, check if they start with a greeting pattern
                # But only if they don't contain question words
                question_words = ["how", "what", "why", "when", "where", "who", "which", "can", "do", "is", "are"]
                if not any(word in query.split() for word in question_words):
                    if any(query.startswith(pattern + " ") for pattern in patterns):
                        return pattern_type

        return None

    def get_conversational_response(self, pattern_type):
        """Get a response for a conversational query."""
        import random
        if pattern_type in self.responses:
            return random.choice(self.responses[pattern_type])
        return None

    def rewrite_query_with_llm(self, user_query):
        """Use the LLM to rewrite the query considering conversation context."""
        if len(self.conversation_history) <= 1:
            return user_query

        # Format recent conversation history (limited to fewer turns to reduce input length)
        recent_history = self.conversation_history[-min(4, len(self.conversation_history)-1):-1]
        history_text = ""
        for msg in recent_history:
            role = "User" if msg["role"] == "user" else "Assistant"
            history_text += f"{role}: {msg['content']}\n"

        # Shorter, more concise prompt for query rewriting
        prompt = f"""Conversation history:
{history_text}
User's latest query: "{user_query}"
Rewrite this as a standalone search query that captures the full intent:"""

        try:
            # Get rewritten query from LLM with increased max_length and max_new_tokens
            rewritten = generate_from_llm(
                prompt,
                max_length=512,      # Increased from 256
                max_new_tokens=128,  # Added this parameter
                pad_token_id=2       # Explicitly set pad_token_id
            )

            # If the rewritten query is too long or empty, fall back to original
            if not rewritten or len(rewritten) > 200 or len(rewritten) < len(user_query) / 2:
                return user_query

            return rewritten
        except Exception as e:
            print(f"Error rewriting query: {e}")
            return user_query  # Fall back to original query

    def generate_answer(self, user_query):
        """Generate a comprehensive response to the user query using all retrieved contexts."""
        # Add the user's message to history
        self.add_message("user", user_query)

        # First check if this is a simple conversational query
        pattern_type = self.is_conversational_query(user_query)
        if pattern_type:
            response = self.get_conversational_response(pattern_type)
            if response:
                self.add_message("assistant", response)
                return response, " "
                # return response, "\n\nSources:\n(Conversational response, no sources needed)"

        # If not a conversational query, proceed with document retrieval
        # Rewrite the query using LLM to incorporate context
        rewritten_query = self.rewrite_query_with_llm(user_query)
        print(f"Original query: '{user_query}'")
        print(f"Rewritten query: '{rewritten_query}'")

        # Search for relevant documents, filtering out those with distance > 1.0
        retrieved_contexts = search_corpus(rewritten_query, self.index, self.texts, self.urls, top_k=3, max_distance=1.0)

        if not retrieved_contexts:
            refusal_message = "I'm sorry, I can't answer that question."
            self.add_message("assistant", refusal_message)
            return refusal_message, " "
            # return refusal_message, "\n\nSources:\n(No relevant sources found)"

        # Debug: Print retrieved contexts with scores
        print("\nRetrieved contexts:")
        for i, ctx in enumerate(retrieved_contexts):
            print(f"Context {i+1} (distance: {ctx['distance']:.4f}): {ctx['context']} ({ctx['url']})")

        # Format retrieved contexts and sources for the LLM prompt
        contexts_formatted = []
        for i, ctx in enumerate(retrieved_contexts):
            # Format each context with its URL source
            contexts_formatted.append(f"Context {i+1} [Source: {ctx['url']}]:\n{ctx['context']}")

        # Join all formatted contexts with clear separators
        all_contexts = "\n\n" + "\n\n".join(contexts_formatted)

        # Create sources list for the return value
        # sources = "\n".join([f"{i+1}. {ctx['url']}" for i, ctx in enumerate(retrieved_contexts)])
        sources = "\n".join([f"{i+1}. {url}" for i, url in enumerate(set(ctx['url'] for ctx in retrieved_contexts))])

        # Format conversation history
        history_text = self.format_conversation_history()

        # Create a comprehensive prompt for the LLM
        prompt = f"""You are an intelligent assistant answering questions based on retrieved information and conversation history.

    TASK: Create a comprehensive, well-structured answer to the user's question using ONLY information from the provided contexts.

    USER QUESTION: {user_query}

    RETRIEVED INFORMATION:
    {all_contexts}

    CONVERSATION HISTORY:
    {history_text}

    INSTRUCTIONS FOR GENERATING YOUR ANSWER:
    1. Format your response appropriately with paragraph breaks for readability
    2. Provide a complete answer that addresses all aspects of the question
    3. Structure your answer with clear paragraphs and logical organization
    4. Be conversational and helpful in your tone
    5. If the retrieved information doesn't fully address the question, acknowledge the limitations
    6. Do not introduce information that isn't supported by the retrieved contexts
    7. Do not repeat the same information multiple times
    8. Prefer information from contexts with lower distance scores (they are more relevant)

    YOUR ANSWER:"""

        try:
            # In production, this would be the actual LLM call:
            response = generate_from_llm(
                prompt,
                max_length=1024,
                max_new_tokens=512,
                pad_token_id=2
            )

            # For testing purposes, we'll use a simple function to simulate an LLM response
            # that integrates information from all contexts
            # response = self.simulate_llm_response(user_query, retrieved_contexts)

            # Add the response to the conversation history
            self.add_message("assistant", response)

            # Return the response and sources
            return response, f"\n\nSources:\n{sources}"

        except Exception as e:
            error_message = f"I'm sorry, I encountered an error while generating a response: {str(e)}"
            self.add_message("assistant", error_message)
            return error_message, "\n\nSources:\n" + sources

In [ ]:
# Build the FAISS index from our expanded knowledge base
index, texts, urls = build_faiss_index(UTD_Knowledge_Base)

# Testing Search Function

In [ ]:
query = 'what is the rank of jindal'
search_corpus(query, index, texts, urls, top_k=5, max_distance=1.0)


# Gradio Interface

In [ ]:
chatbot = ConversationalChatbot(index, texts, urls)

In [ ]:
chatbot.conversation_history = []

In [ ]:
!pip install gradio -qq

In [ ]:
# import gradio as gr
# import textwrap
# import re
# from datetime import datetime

# # Define the Gradio interface
# def add_message(history, message, role="user"):
#     """Add a message to the chat history."""
#     timestamp = datetime.now().strftime("%H:%M")
#     history.append((role, message, timestamp))
#     return history

# def process_query(history, query):
#     """Process the query using the get_answer function and update the chat history."""
#     if not query.strip():
#         return history, ""

#     # Add user message to history
#     history = add_message(history, query, "user")

#     # Process the query
#     response = get_answer(chatbot, query)

#     # Process response to remove duplicate source references
#     response = remove_duplicate_sources(response)

#     # Add bot message to history
#     history = add_message(history, response, "bot")

#     return history, ""

# def remove_duplicate_sources(text):
#     """Remove duplicate source references from the text."""
#     import re

#     # Find all source references using regex
#     source_pattern = r'\[(.*?)\]'
#     sources = re.findall(source_pattern, text)

#     # If there are no sources or just one, return the original text
#     if len(sources) <= 1:
#         return text

#     # Track seen sources and their first occurrence position
#     seen_sources = {}
#     for match in re.finditer(source_pattern, text):
#         source = match.group(1)
#         if source not in seen_sources:
#             seen_sources[source] = match.span()

#     # Build a new text with only the first occurrence of each source
#     result = ""
#     last_end = 0

#     # Sort the spans by start position to maintain order
#     sorted_spans = sorted(seen_sources.values())

#     for start, end in sorted_spans:
#         result += text[last_end:start]
#         result += text[start:end]
#         last_end = end

#     # Add remaining text after the last source
#     result += text[last_end:]

#     return result

# def format_message(message):
#     """Format the message for display."""
#     role, text, timestamp = message

#     if role == "user":
#         return f"""
#         <div style="display: flex; flex-direction: row; justify-content: flex-end; margin-bottom: 10px;">
#             <div style="background-color: #DCF8C6; border-radius: 15px; padding: 10px 15px; max-width: 80%; position: relative;">
#                 <p style="margin: 0; white-space: pre-wrap;">{text}</p>
#                 <span style="font-size: 0.7em; color: #888; position: absolute; bottom: 5px; right: 10px;">{timestamp}</span>
#             </div>
#         </div>
#         """
#     else:  # bot
#         return f"""
#         <div style="display: flex; flex-direction: row; margin-bottom: 10px;">
#             <div style="background-color: #F1F0F0; border-radius: 15px; padding: 10px 15px; max-width: 80%; position: relative;">
#                 <p style="margin: 0; white-space: pre-wrap;">{text}</p>
#                 <span style="font-size: 0.7em; color: #888; position: absolute; bottom: 5px; right: 10px;">{timestamp}</span>
#             </div>
#         </div>
#         """

# def render_chat_history(history):
#     """Render the chat history as HTML."""
#     messages_html = "".join([format_message(message) for message in history])
#     return f"""
#     <div style="height: 600px; overflow-y: auto; padding: 15px; background-color: #E5DDD5;">
#         {messages_html}
#     </div>
#     """

# # Custom CSS for styling the chat interface to match Signal's design
# css = """
# .message-container {
#     padding: 10px;
#     margin-bottom: 10px;
#     border-radius: 18px;
#     max-width: 80%;
#     position: relative;
# }

# .user-message {
#     background-color: #3A76F0;
#     color: white;
#     margin-left: auto;
#     margin-right: 10px;
# }

# .bot-message {
#     background-color: #2C2C2E;
#     color: white;
#     margin-right: auto;
#     margin-left: 10px;
# }

# .timestamp {
#     font-size: 10px;
#     color: rgba(255, 255, 255, 0.7);
#     position: absolute;
#     bottom: 5px;
#     right: 10px;
# }

# .message-content {
#     padding-right: 40px;
#     word-wrap: break-word;
#     white-space: pre-wrap;  /* This preserves line breaks */
# }

# .chatbox {
#     border-radius: 10px;
#     border: 1px solid #1B1B1D;
#     background-color: #161618;
#     height: 500px;
#     overflow-y: auto;
# }

# .chat-header {
#     background-color: #1B1B1D;
#     color: white;
#     padding: 15px;
#     border-radius: 10px 10px 0 0;
#     font-weight: bold;
#     font-size: 18px;
#     display: flex;
#     align-items: center;
# }

# .profile-pic {
#     width: 40px;
#     height: 40px;
#     border-radius: 50%;
#     background-color: #3A76F0;
#     margin-right: 10px;
#     display: flex;
#     align-items: center;
#     justify-content: center;
#     color: white;
#     font-weight: bold;
# }

# .chat-footer {
#     background-color: #1B1B1D;
#     padding: 10px;
#     border-radius: 0 0 10px 10px;
#     display: flex;
#     align-items: center;
# }

# .send-button {
#     background-color: #3A76F0;
#     color: white;
#     border-radius: 50%;
#     width: 40px;
#     height: 40px;
#     display: flex;
#     align-items: center;
#     justify-content: center;
#     cursor: pointer;
#     margin-left: 10px;
# }

# .input-message {
#     flex-grow: 1;
#     border-radius: 20px;
#     padding: 10px;
#     border: none;
#     background-color: #2C2C2E;
#     color: white;
# }
# """

# # Custom function to render chat history with timestamps
# def format_chat_history(history):
#     html = ""
#     for role, message, timestamp in history:
#         if role == "user":
#             html += f"""
#             <div class="message-container user-message">
#                 <div class="message-content">{message}</div>
#                 <div class="timestamp">{timestamp}</div>
#             </div>
#             """
#         else:
#             # Process the bot message to make links clickable first
#             import re

#             # First make URLs clickable by finding URL patterns and replacing them with anchor tags
#             url_pattern = r'(https?://[^\s\n]+)'
#             formatted_message = re.sub(
#                 url_pattern,
#                 r'<a href="\1" target="_blank" style="color: #4DA3FF; text-decoration: underline;">\1</a>',
#                 message
#             )

#             # Then look for text that appears to be a source reference
#             source_pattern = r'\[(.*?)\]'
#             formatted_message = re.sub(
#                 source_pattern,
#                 r'<span style="color: #4DA3FF; cursor: pointer; text-decoration: underline;" onclick="alert(\'Source: \1\')">[\1]</span>',
#                 formatted_message
#             )

#             # Finally replace newlines with <br> tags after all other processing
#             formatted_message = formatted_message.replace("\n", "<br>")

#             html += f"""
#             <div class="message-container bot-message">
#                 <div class="message-content">{formatted_message}</div>
#                 <div class="timestamp">{timestamp}</div>
#             </div>
#             """
#     return html

# # Main Gradio app
# with gr.Blocks(css=css) as demo:
#     with gr.Column():
#         # Chat header with Signal styling
#         with gr.Row(elem_classes="chat-header"):
#             gr.HTML('<div>UTD Chatbot</div>')

#         # Chat history
#         chat_history = gr.State([])
#         chat_display = gr.HTML(elem_classes="chatbox")

#         # Input area
#         with gr.Row(elem_classes="chat-footer"):
#             with gr.Column(scale=6):
#                 msg = gr.Textbox(
#                     show_label=False,
#                     placeholder="Type a message...",
#                     elem_classes="input-message"
#                 )
#             with gr.Column(scale=1):
#                 send = gr.Button("➤", elem_classes="send-button")

#         # Update chat display when new messages are added
#         def update_chat(history):
#             return format_chat_history(history)

#         # Handle sending messages
#         def on_submit(message, history):
#             if message.strip() == "":
#                 return history, "", update_chat(history)

#             # Process the message and update history
#             new_history, _ = process_query(history, message)

#             return new_history, "", update_chat(new_history)

#         # Connect the components
#         send.click(
#             on_submit,
#             [msg, chat_history],
#             [chat_history, msg, chat_display]
#         )
#         msg.submit(
#             on_submit,
#             [msg, chat_history],
#             [chat_history, msg, chat_display]
#         )

# chatbot.conversation_history = []
# if __name__ == "__main__":
#     demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6d5cf5e58f8c984522.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
import textwrap
import re
from datetime import datetime

# Define the Gradio interface
def add_message(history, message, role="user"):
    """Add a message to the chat history."""
    timestamp = datetime.now().strftime("%H:%M")
    history.append((role, message, timestamp))
    return history

def process_query(history, query):
    """Process the query using the get_answer function and update the chat history."""
    if not query.strip():
        return history, ""

    # Add user message to history
    history = add_message(history, query, "user")

    # Process the query
    response = get_answer(chatbot, query)

    # Process response to remove duplicate source references
    response = remove_duplicate_sources(response)

    # Add bot message to history
    history = add_message(history, response, "bot")

    return history, ""

def remove_duplicate_sources(text):
    """Remove duplicate source references from the text."""
    import re

    # Find all source references using regex
    source_pattern = r'\[(.*?)\]'
    sources = re.findall(source_pattern, text)

    # If there are no sources or just one, return the original text
    if len(sources) <= 1:
        return text

    # Track seen sources and their first occurrence position
    seen_sources = {}
    for match in re.finditer(source_pattern, text):
        source = match.group(1)
        if source not in seen_sources:
            seen_sources[source] = match.span()

    # Build a new text with only the first occurrence of each source
    result = ""
    last_end = 0

    # Sort the spans by start position to maintain order
    sorted_spans = sorted(seen_sources.values())

    for start, end in sorted_spans:
        result += text[last_end:start]
        result += text[start:end]
        last_end = end

    # Add remaining text after the last source
    result += text[last_end:]

    return result

def format_message(message):
    """Format the message for display."""
    role, text, timestamp = message

    if role == "user":
        return f"""
        <div style="display: flex; flex-direction: row; justify-content: flex-end; margin-bottom: 10px;">
            <div style="background-color: #DCF8C6; border-radius: 15px; padding: 10px 15px; max-width: 80%; position: relative;">
                <p style="margin: 0; white-space: pre-wrap;">{text}</p>
                <span style="font-size: 0.7em; color: #888; position: absolute; bottom: 5px; right: 10px;">{timestamp}</span>
            </div>
        </div>
        """
    else:  # bot
        return f"""
        <div style="display: flex; flex-direction: row; margin-bottom: 10px;">
            <div style="background-color: #F1F0F0; border-radius: 15px; padding: 10px 15px; max-width: 80%; position: relative;">
                <p style="margin: 0; white-space: pre-wrap;">{text}</p>
                <span style="font-size: 0.7em; color: #888; position: absolute; bottom: 5px; right: 10px;">{timestamp}</span>
            </div>
        </div>
        """

def render_chat_history(history):
    """Render the chat history as HTML."""
    messages_html = "".join([format_message(message) for message in history])
    return f"""
    <div style="height: 600px; overflow-y: auto; padding: 15px; background-color: #E5DDD5;">
        {messages_html}
    </div>
    """

# Custom CSS for styling the chat interface to match Signal's design
css = """
.message-container {
    padding: 10px;
    margin-bottom: 10px;
    border-radius: 18px;
    max-width: 80%;
    position: relative;
}

.user-message {
    background-color: #3A76F0;
    color: white;
    margin-left: auto;
    margin-right: 10px;
}

.bot-message {
    background-color: #2C2C2E;
    color: white;
    margin-right: auto;
    margin-left: 10px;
}

.timestamp {
    font-size: 10px;
    color: rgba(255, 255, 255, 0.7);
    position: absolute;
    bottom: 5px;
    right: 10px;
}

.message-content {
    padding-right: 40px;
    word-wrap: break-word;
    white-space: pre-wrap;  /* This preserves line breaks */
}

.chatbox {
    border-radius: 10px;
    border: 1px solid #1B1B1D;
    background-color: #161618;
    height: 500px;
    overflow-y: auto;
}

.chat-header {
    background-color: #1B1B1D;
    color: white;
    padding: 15px;
    border-radius: 10px 10px 0 0;
    font-weight: bold;
    font-size: 18px;
    display: flex;
    align-items: center;
    justify-content: space-between;
}

.profile-pic {
    width: 40px;
    height: 40px;
    border-radius: 50%;
    background-color: #3A76F0;
    margin-right: 10px;
    display: flex;
    align-items: center;
    justify-content: center;
    color: white;
    font-weight: bold;
}

.chat-footer {
    background-color: #1B1B1D;
    padding: 10px;
    border-radius: 0 0 10px 10px;
    display: flex;
    align-items: center;
}

.send-button {
    background-color: #3A76F0;
    color: white;
    border-radius: 50%;
    width: 40px;
    height: 40px;
    display: flex;
    align-items: center;
    justify-content: center;
    cursor: pointer;
    margin-left: 10px;
}

.input-message {
    flex-grow: 1;
    border-radius: 20px;
    padding: 10px;
    border: none;
    background-color: #2C2C2E;
    color: white;
}

.fresh-search-button {
    background-color: #4CAF50;
    color: white;
    border: none;
    padding: 8px 15px;
    border-radius: 15px;
    cursor: pointer;
    font-size: 14px;
    transition: background-color 0.3s;
}

.fresh-search-button:hover {
    background-color: #45a049;
}
"""

# Custom function to render chat history with timestamps
def format_chat_history(history):
    html = ""
    for role, message, timestamp in history:
        if role == "user":
            html += f"""
            <div class="message-container user-message">
                <div class="message-content">{message}</div>
                <div class="timestamp">{timestamp}</div>
            </div>
            """
        else:
            # Process the bot message to make links clickable first
            import re

            # First make URLs clickable by finding URL patterns and replacing them with anchor tags
            url_pattern = r'(https?://[^\s\n]+)'
            formatted_message = re.sub(
                url_pattern,
                r'<a href="\1" target="_blank" style="color: #4DA3FF; text-decoration: underline;">\1</a>',
                message
            )

            # Then look for text that appears to be a source reference
            source_pattern = r'\[(.*?)\]'
            formatted_message = re.sub(
                source_pattern,
                r'<span style="color: #4DA3FF; cursor: pointer; text-decoration: underline;" onclick="alert(\'Source: \1\')">[\1]</span>',
                formatted_message
            )

            # Finally replace newlines with <br> tags after all other processing
            formatted_message = formatted_message.replace("\n", "<br>")

            html += f"""
            <div class="message-container bot-message">
                <div class="message-content">{formatted_message}</div>
                <div class="timestamp">{timestamp}</div>
            </div>
            """
    return html

# Function to handle fresh search
def fresh_search():
    # Reset conversation history
    chatbot.conversation_history = []
    # Return empty history and update chat display
    return [], update_chat([])

# Main Gradio app
with gr.Blocks(css=css) as demo:
    with gr.Column():
        # Chat header with Signal styling and new Fresh Search button
        with gr.Row(elem_classes="chat-header"):
            gr.HTML('<div>UTD Chatbot</div>')
            fresh_search_btn = gr.Button("Clear", elem_classes="fresh-search-button")

        # Chat history
        chat_history = gr.State([])
        chat_display = gr.HTML(elem_classes="chatbox")

        # Input area
        with gr.Row(elem_classes="chat-footer"):
            with gr.Column(scale=6):
                msg = gr.Textbox(
                    show_label=False,
                    placeholder="Type a message...",
                    elem_classes="input-message"
                )
            with gr.Column(scale=1):
                send = gr.Button("➤", elem_classes="send-button")

        # Update chat display when new messages are added
        def update_chat(history):
            return format_chat_history(history)

        # Handle sending messages
        def on_submit(message, history):
            if message.strip() == "":
                return history, "", update_chat(history)

            # Process the message and update history
            new_history, _ = process_query(history, message)

            return new_history, "", update_chat(new_history)

        # Connect the components
        send.click(
            on_submit,
            [msg, chat_history],
            [chat_history, msg, chat_display]
        )
        msg.submit(
            on_submit,
            [msg, chat_history],
            [chat_history, msg, chat_display]
        )

        # Connect fresh search button
        fresh_search_btn.click(
            fresh_search,
            [],
            [chat_history, chat_display]
        )



In [ ]:
# demo.close()

Closing server running on port: 7869


In [ ]:
chatbot.conversation_history = []
if __name__ == "__main__":
    demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://09d02d2b2c269dbc5c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Clear History

In [ ]:
chatbot.conversation_history = []

# Test Retrieval and Generation

In [ ]:
chatbot.conversation_history = []
print(get_answer(chatbot, "ranking of jindal"))

Original query: 'ranking of jindal'
Rewritten query: 'What is the ranking of Jindal in India?"'
NEAR MATCH FOUND: The number of universities from the United States and Canada in the top 100 of the global ranking ha...
NEAR MATCH FOUND: Institute for Excellence in Corporate Governance
International Center for Decision and Risk Analysis...
NEAR MATCH FOUND: Message from the Dean
Naveen Jindal
Rankings
Student Experience
Outcomes & Success Factors
MANAGEMEN...

Match statistics: 0 exact, 3 near, 0 vector-only
All results have distance <= 1.0

Retrieved contexts:
Context 1 (distance: 0.5000): The number of universities from the United States and Canada in the top 100 of the global ranking has steadily decreased since 2005. However, productivity in both countries has steadily increased. Top 100 U.S. universities produced 7,957 articles in the 2014-2018 reporting period, compared to 5,359 articles in the inaugural 2000-2004 period. The number of articles from Canadian universities in the Top